In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr').getOrCreate()

In [0]:
data = spark.read.options(header=True, inferSchema=True).csv('/FileStore/tables/titanic.csv')
data.show(n=5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

In [0]:
data.columns

Out[3]: ['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [0]:
data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [0]:
my_cols = data.select(['Survived','Pclass','Sex',
 'Age',
 'SibSp',
 'Parch','Fare','Embarked'])

In [0]:
#missing values
my_final_data = my_cols.na.drop()

In [0]:
#working with categorical column
from pyspark.ml.feature import VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer


In [0]:
#string indexer allow us to convert every string into a number
gender_indexer = StringIndexer(inputCol='Sex', outputCol='SexIndex')
# A B C --> string
# 0 1 2 -->string index
# one hot encoding above

In [0]:
gender_encoder = OneHotEncoder(inputCol='SexIndex', outputCol='SexVec')


In [0]:
embark_indexer = StringIndexer(inputCol='Embarked', outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex', outputCol='EmbarkVec')

In [0]:
#ceate an assember
assembler = VectorAssembler(inputCols=['Pclass', 'SexVec', 'EmbarkVec', 'Age', 'SibSp', 'Parch','Fare'], 
                           outputCol='features')



In [0]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

#setting up the stages using pipeline
log_reg = LogisticRegression(featuresCol='features', labelCol='Survived')
pipeline = Pipeline(stages=[gender_indexer, embark_indexer, gender_encoder, embark_encoder, assembler, log_reg]) #list of everything we wanna do step by step

In [0]:
train_data, test_data = my_final_data.randomSplit([0.7, 0.3])
fit_model = pipeline.fit(train_data)

In [0]:
results = fit_model.transform(test_data)


In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Survived')
my_eval.evaluate(results)

Out[16]: 0.7119047619047619

In [0]:
results.select('Survived', 'prediction').show(n=5)

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
+--------+----------+
only showing top 5 rows

